In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
%cd "../.."

/home/hazzu/Code/opencv_streamlit


# 1. Define function


In [3]:
import os
import cv2
import numpy as np
from typing import Tuple, List
from services.semantic_keypoint_detection.services import DATATYPES


def read_image(type: int, name: str):
    """
    Reads an image and its ground truth keypoints from the specified dataset.

    :param type: The type of the dataset.
    :param name: The name of the image.

    :return (image, ground_truth): A tuple containing the image and the ground truth keypoints.
    """
    image = cv2.imread(os.path.join(DATATYPES[type], "images", f"{name}.png"))
    ground_truth = np.load(os.path.join(DATATYPES[type], "points", f"{name}.npy"))
    ground_truth = [cv2.KeyPoint(y, x, 1, 0, 0, 0) for x, y in ground_truth]
    return (image, ground_truth)


def rotate_image(image: cv2.typing.MatLike, angle: int) -> cv2.typing.MatLike:
    """
    Rotates an image by a given angle.

    :param image: The image to rotate.
    :param angle: The angle to rotate the image by.

    :return rotated_image: The rotated image.
    """
    h, w = image.shape[:2]
    matrix_rotation = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, matrix_rotation, (w, h))
    return rotated_image


def rotate_keypoints(
    size: Tuple[int, int], keypoints: List[cv2.KeyPoint], angle: int
) -> List[cv2.KeyPoint]:
    """
    Rotates the keypoints of an image.

    :param size: The size of the image (witdh, height).
    :param keypoints: The keypoints to rotate.
    :param angle: The angle to rotate the keypoints by.

    :return (result, idx): A tuple containing the rotated keypoints and their indices in the original list.
    """
    matrix_rotation = cv2.getRotationMatrix2D((size[0] / 2, size[1] / 2), angle, 1)
    kps = np.array([[kp.pt[0], kp.pt[1]] for kp in keypoints])
    kps = np.concatenate([kps, np.ones((len(kps), 1))], axis=1)
    rotated_kps = np.array(np.dot(matrix_rotation, kps.T)).T

    result, idx = [], []
    for i in range(len(rotated_kps)):
        kp = rotated_kps[i]
        if 0 <= kp[0] < size[0] and 0 <= kp[1] < size[1]:
            result.append(cv2.KeyPoint(kp[0], kp[1], 1, 0, 0, 0))
            idx.append(i)

    return (result, idx)

# 2. Cache descriptors of original image


In [4]:
import cv2
from tqdm import tqdm
from services.keypoint_matching.superpoint import SuperPointFrontend


def cache_descriptors(model, convert=False):
    """
    Caches the descriptors of the images in the datasets.

    :param model: The model to use for the caching.

    :return cache_descriptors: The cached descriptors.
    """
    cache_descriptors = []

    for i in range(8):
        cache_descriptors.append([])

        for j in tqdm(range(500), desc=f"Cache descriptors for shape type {i}"):
            image, ground_truth = read_image(i, str(j))
            gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            if convert:
                gray_scale = gray_scale.astype(np.float32) / 255.0

            if len(ground_truth) > 0:
                descriptors = model.compute(gray_scale, ground_truth)[1]
            else:
                descriptors = None

            if descriptors is not None:
                assert descriptors.shape[0] == len(
                    ground_truth
                ), f"{descriptors.shape[0]} != {len(ground_truth)}"
            else:
                assert len(ground_truth) == 0

            cache_descriptors[i].append(descriptors)

    return cache_descriptors


print("Caching descriptors for SIFT...")
sift = cv2.SIFT_create()
cache_sift_descriptors = cache_descriptors(sift)

print("Caching descriptors for ORB...")
orb = cv2.ORB_create(edgeThreshold=0, fastThreshold=0)
cache_orb_descriptors = cache_descriptors(orb)

print("Caching descriptors for SuperPoint...")
fe = SuperPointFrontend(
    weights_path="./services/keypoint_matching/superpoint_v1.pth",
    nms_dist=4,
    conf_thresh=0.015,
    nn_thresh=0.7,
    cuda=True,
)
cache_superpoint_descriptors = cache_descriptors(fe, True)

Caching descriptors for SIFT...


Cache descriptors for shape type 7: 100%|██████████| 500/500 [00:00<00:00, 1721.86it/s]


Caching descriptors for ORB...


Cache descriptors for shape type 7: 100%|██████████| 500/500 [00:00<00:00, 3446.96it/s]
/home/hazzu/Code/opencv_streamlit/services/keypoint_matching/superpoint.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues 

Caching descriptors for SuperPoint...
Run on GPU


Cache descriptors for shape type 0:   0%|          | 0/500 [00:00<?, ?it/s]/home/hazzu/anaconda3/envs/opencv_streamlit/lib/python3.12/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
Cache descriptors for shape type 7: 100%|██████████| 500/500 [00:01<00:00, 393.30it/s]


# 3. Get accuracy matching keypoints of SIFT


In [5]:
sift = cv2.SIFT_create()
bf_sift = cv2.BFMatcher(cv2.NORM_L2)
accuracy_matches_sift = []

for angle in range(0, 360, 10):
    accuracy_matches_sift.append([])

    for type in range(8):
        accuracy_matches_sift[-1].append([])

        for i in tqdm(range(500), f"Angle: {angle} - Shape type: {type}"):
            image, ground_truth = read_image(type, str(i))
            gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            descriptor = cache_sift_descriptors[type][i]

            if descriptor is None:
                continue

            # Rotate the image and the keypoints.
            rotated_image = rotate_image(gray_scale, angle)
            rotated_keypoints, idx = rotate_keypoints(
                (image.shape[1], image.shape[0]), ground_truth, angle
            )

            # Compute the descriptors of the rotated image.
            rotated_descriptor = sift.compute(rotated_image, rotated_keypoints)[1]
            if rotated_descriptor is not None:
                assert len(rotated_descriptor) == len(rotated_keypoints)
            else:
                assert len(rotated_keypoints) == 0
                continue

            # Get the original keypoints and descriptors.
            original_descriptor = np.array(descriptor)[idx]

            # Match the descriptors of the original and rotated images.
            if len(rotated_descriptor) == 1:
                accuracy_matches_sift[-1][-1].append(1)
            else:
                matches = bf_sift.knnMatch(rotated_descriptor, original_descriptor, k=2)
                matches = [
                    m
                    for m, n in matches
                    if m.distance < 0.75 * n.distance and m.trainIdx == m.queryIdx
                ]
                accuracy_matches_sift[-1][-1].append(
                    len(matches) / len(original_descriptor)
                )

        if len(accuracy_matches_sift[-1][-1]) > 0:
            print(f"Accuracy: {np.mean(accuracy_matches_sift[-1][-1])}")
        else:
            print("No matches found.")

Angle: 0 - Shape type: 0: 100%|██████████| 500/500 [00:00<00:00, 950.41it/s] 


Accuracy: 1.0


Angle: 0 - Shape type: 1: 100%|██████████| 500/500 [00:00<00:00, 1096.96it/s]


Accuracy: 1.0


Angle: 0 - Shape type: 2: 100%|██████████| 500/500 [00:00<00:00, 3231.35it/s]


...

Accuracy: 0.9942333333333333


Angle: 350 - Shape type: 6: 100%|██████████| 500/500 [00:00<00:00, 1052.85it/s]


Accuracy: 0.9625


Angle: 350 - Shape type: 7: 100%|██████████| 500/500 [00:00<00:00, 1189.48it/s]

Accuracy: 0.7321881317335863


In [6]:
for i in range(36):
    print(f"Angle: {i * 10}")
    flatten = []
    for j in range(8):
        flatten += accuracy_matches_sift[i][j]
    print(f"Accuracy: {np.mean(flatten)}")

Angle: 0
Accuracy: 0.99940581098872
Angle: 10
Accuracy: 0.7881595546809593
Angle: 20
Accuracy: 0.5490263526140332
Angle: 30
Accuracy: 0.3204361123232021
Angle: 40
Accuracy: 0.1487801533445735
Angle: 50
Accuracy: 0.07437489233027887
Angle: 60
Accuracy: 0.04612020646309362
Angle: 70
Accuracy: 0.034927788098986355
Angle: 80
Accuracy: 0.03580697704764605
Angle: 90
Accuracy: 0.03778066947842803
Angle: 100
Accuracy: 0.03602597575878873
Angle: 110
Accuracy: 0.03155945189949225
Angle: 120
Accuracy: 0.032444348389885455
Angle: 130
Accuracy: 0.029572338489535943
Angle: 140
Accuracy: 0.026749381805561582
Angle: 150
Accuracy: 0.02254348343217012
Angle: 160
Accuracy: 0.020198264538142344
Angle: 170
Accuracy: 0.019481059341679725
Angle: 180
Accuracy: 0.020268073875191923
Angle: 190
Accuracy: 0.021438548390627105
Angle: 200
Accuracy: 0.02165277818916633
Angle: 210
Accuracy: 0.02454802684294872
Angle: 220
Accuracy: 0.028788643917254025
Angle: 230
Accuracy: 0.031119166510674104
Angle: 240
Accuracy: 0.0

# 4. Get accuracy matching keypoints of ORB


In [9]:
orb = cv2.ORB_create(edgeThreshold=0, fastThreshold=0)
bf_orb = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
accuracy_matches_orb = []

for angle in range(0, 360, 10):
    accuracy_matches_orb.append([])

    for type in range(8):
        accuracy_matches_orb[-1].append([])

        for i in tqdm(range(500), f"Angle: {angle} - Shape type: {type}"):
            image, ground_truth = read_image(type, str(i))
            gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            descriptor = cache_orb_descriptors[type][i]

            if descriptor is None:
                continue

            # Rotate the image and the keypoints.
            rotated_image = rotate_image(gray_scale, angle)
            rotated_keypoints, idx = rotate_keypoints(
                (image.shape[1], image.shape[0]), ground_truth, angle
            )

            # Compute the descriptors of the rotated image.
            rotated_descriptor = orb.compute(rotated_image, rotated_keypoints)[1]
            if rotated_descriptor is not None:
                assert len(rotated_descriptor) == len(rotated_keypoints)
            else:
                assert len(rotated_keypoints) == 0
                continue

            # Get the original keypoints and descriptors.
            original_descriptor = np.array(descriptor)[idx]

            # Match the descriptors of the original and rotated images.
            if len(rotated_descriptor) == 1:
                accuracy_matches_orb[-1][-1].append(1)
            else:
                matches = bf_orb.match(rotated_descriptor, original_descriptor)
                matches = [m for m in matches if m.trainIdx == m.queryIdx]
                accuracy_matches_orb[-1][-1].append(
                    len(matches) / len(original_descriptor)
                )

        if len(accuracy_matches_orb[-1][-1]) > 0:
            print(f"Accuracy: {np.mean(accuracy_matches_orb[-1][-1])}")
        else:
            print("No matches found.")

Angle: 0 - Shape type: 0:   0%|          | 0/500 [00:00<?, ?it/s]

Angle: 0 - Shape type: 0: 100%|██████████| 500/500 [00:00<00:00, 1547.82it/s]


Accuracy: 0.9994561736690857


Angle: 0 - Shape type: 1: 100%|██████████| 500/500 [00:00<00:00, 1781.94it/s]


Accuracy: 1.0


...

Accuracy: 0.9913


Angle: 350 - Shape type: 6: 100%|██████████| 500/500 [00:00<00:00, 1738.41it/s]


Accuracy: 0.9953


Angle: 350 - Shape type: 7: 100%|██████████| 500/500 [00:00<00:00, 2293.66it/s]

Accuracy: 0.8457272777727324


In [10]:
for i in range(36):
    print(f"Angle: {i * 10}")
    flatten = []
    for j in range(8):
        flatten += accuracy_matches_orb[i][j]
    print(f"Accuracy: {np.mean(flatten)}")

Angle: 0
Accuracy: 0.9996223863455721
Angle: 10
Accuracy: 0.8904585558897983
Angle: 20
Accuracy: 0.7001511527266256
Angle: 30
Accuracy: 0.39835639133403716
Angle: 40
Accuracy: 0.23544552022054208
Angle: 50
Accuracy: 0.16145194048092804
Angle: 60
Accuracy: 0.1305645378651016
Angle: 70
Accuracy: 0.11667570371269598
Angle: 80
Accuracy: 0.1114618020463434
Angle: 90
Accuracy: 0.1084289350336079
Angle: 100
Accuracy: 0.09795824060050774
Angle: 110
Accuracy: 0.08971309645150964
Angle: 120
Accuracy: 0.08370406936850529
Angle: 130
Accuracy: 0.08264135214361899
Angle: 140
Accuracy: 0.0904216079147379
Angle: 150
Accuracy: 0.09474493103735496
Angle: 160
Accuracy: 0.10228440214799132
Angle: 170
Accuracy: 0.11620761522558028
Angle: 180
Accuracy: 0.13170642944033528
Angle: 190
Accuracy: 0.1148340809076005
Angle: 200
Accuracy: 0.1029375944221676
Angle: 210
Accuracy: 0.09486399306401053
Angle: 220
Accuracy: 0.0933793222045747
Angle: 230
Accuracy: 0.0914450438270937
Angle: 240
Accuracy: 0.091635183632381

# 5. Get accuracy matching keypoints of SuperPoint


In [11]:
from services.keypoint_matching.superpoint import SuperPointFrontend

fe = SuperPointFrontend(
    weights_path="./services/keypoint_matching/superpoint_v1.pth",
    nms_dist=4,
    conf_thresh=0.015,
    nn_thresh=0.7,
    cuda=True,
)
bf_sift = cv2.BFMatcher(cv2.NORM_L2)
accuracy_matches_superpoint = []

for angle in range(0, 360, 10):
    accuracy_matches_superpoint.append([])

    for type in range(8):
        accuracy_matches_superpoint[-1].append([])

        for i in tqdm(range(500), f"Angle: {angle} - Shape type: {type}"):
            image, ground_truth = read_image(type, str(i))
            gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray_scale = gray_scale.astype(np.float32) / 255.0
            descriptor = cache_superpoint_descriptors[type][i]

            if descriptor is None:
                continue

            # Rotate the image and the keypoints.
            rotated_image = rotate_image(gray_scale, angle)
            rotated_keypoints, idx = rotate_keypoints(
                (image.shape[1], image.shape[0]), ground_truth, angle
            )

            # Compute the descriptors of the rotated image.
            rotated_descriptor = fe.compute(rotated_image, rotated_keypoints)[1]
            if rotated_descriptor is not None:
                assert len(rotated_descriptor) == len(rotated_keypoints)
            else:
                assert len(rotated_keypoints) == 0
                continue

            # Get the original keypoints and descriptors.
            original_descriptor = np.array(descriptor)[idx]

            # Match the descriptors of the original and rotated images.
            if len(rotated_descriptor) == 1:
                accuracy_matches_superpoint[-1][-1].append(1)
            else:
                matches = bf_sift.knnMatch(rotated_descriptor, original_descriptor, k=2)
                matches = [
                    m
                    for m, n in matches
                    if m.distance < 0.75 * n.distance and m.trainIdx == m.queryIdx
                ]
                accuracy_matches_superpoint[-1][-1].append(
                    len(matches) / len(original_descriptor)
                )

        if len(accuracy_matches_superpoint[-1][-1]) > 0:
            print(f"Accuracy: {np.mean(accuracy_matches_superpoint[-1][-1])}")
        else:
            print("No matches found.")

/home/hazzu/Code/opencv_streamlit/services/keypoint_matching/superpoint.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(weights_path

Run on GPU


Angle: 0 - Shape type: 0:   0%|          | 0/500 [00:00<?, ?it/s]/home/hazzu/anaconda3/envs/opencv_streamlit/lib/python3.12/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
Angle: 0 - Shape type: 0: 100%|██████████| 500/500 [00:02<00:00, 245.06it/s]


Accuracy: 1.0


Angle: 0 - Shape type: 1: 100%|██████████| 500/500 [00:01<00:00, 290.72it/s]


...

Accuracy: 0.9841666666666667


Angle: 350 - Shape type: 6: 100%|██████████| 500/500 [00:00<00:00, -514.13it/s]


Accuracy: 0.9933


Angle: 350 - Shape type: 7: 100%|██████████| 500/500 [00:01<00:00, 404.43it/s]

Accuracy: 0.8316574755211119


In [12]:
for i in range(36):
    print(f"Angle: {i * 10}")
    flatten = []
    for j in range(8):
        flatten += accuracy_matches_superpoint[i][j]
    print(f"Accuracy: {np.mean(flatten)}")

Angle: 0
Accuracy: 1.0
Angle: 10
Accuracy: 0.9378956268344535
Angle: 20
Accuracy: 0.8604612551499848
Angle: 30
Accuracy: 0.6121989346471783
Angle: 40
Accuracy: 0.2912652832943284
Angle: 50
Accuracy: 0.10048076426910854
Angle: 60
Accuracy: 0.0503773490000224
Angle: 70
Accuracy: 0.04058819485603551
Angle: 80
Accuracy: 0.04116957401439487
Angle: 90
Accuracy: 0.044158501975213565
Angle: 100
Accuracy: 0.041555517433632116
Angle: 110
Accuracy: 0.036596040341527705
Angle: 120
Accuracy: 0.035833153231035225
Angle: 130
Accuracy: 0.03439995955919523
Angle: 140
Accuracy: 0.031452963475435386
Angle: 150
Accuracy: 0.026978807860416017
Angle: 160
Accuracy: 0.024628004847423175
Angle: 170
Accuracy: 0.0203092656820278
Angle: 180
Accuracy: 0.020524217724477146
Angle: 190
Accuracy: 0.022365608056624367
Angle: 200
Accuracy: 0.024800514709684685
Angle: 210
Accuracy: 0.028361442178221417
Angle: 220
Accuracy: 0.03371017004963795
Angle: 230
Accuracy: 0.0361486195923282
Angle: 240
Accuracy: 0.0379430343565997

# 6. Convert result to Numpy array


In [13]:
results_sift = []
results_orb = []
results_superpoint = []

for i in range(36):
    flatten = []
    for j in range(8):
        flatten += accuracy_matches_sift[i][j]
    results_sift.append(np.mean(flatten))

    flatten = []
    for j in range(8):
        flatten += accuracy_matches_orb[i][j]
    results_orb.append(np.mean(flatten))

    flatten = []
    for j in range(8):
        flatten += accuracy_matches_superpoint[i][j]
    results_superpoint.append(np.mean(flatten))

np.save("./services/keypoint_matching/results_sift.npy", results_sift)
np.save("./services/keypoint_matching/results_orb.npy", results_orb)
np.save("./services/keypoint_matching/results_superpoint.npy", results_superpoint)